In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("E:/Projects/Smart-Logistics-System/data/processed/dataset_with_risk_levels.csv")

df.head()

,Latitude,Longitude,Inventory_Level,Temperature,Humidity,Precipitation(mm),Waiting_Time,User_Transaction_Amount,User_Purchase_Frequency,Asset_Utilization,...,Asset_ID_Truck_3,Asset_ID_Truck_4,Asset_ID_Truck_5,Asset_ID_Truck_6,Asset_ID_Truck_7,Asset_ID_Truck_8,Asset_ID_Truck_9,delay_probability,ml_risk_level,final_risk_level
0,-65.7383,11.2497,390,27.0,67.8,10.023475,38,320,4,60.1,...,0,0,0,0,1,0,0,0.339254,Low,Low
1,22.2748,-131.7086,491,22.5,54.3,21.397599,16,439,7,80.9,...,0,0,0,1,0,0,0,0.870765,Critical,Critical
2,54.9232,79.5455,190,25.2,62.2,14.520535,34,355,3,99.2,...,0,0,0,0,0,0,0,0.452376,Medium,High
3,42.3900,-1.4788,330,25.4,52.3,2.613761,37,227,5,97.4,...,0,0,0,0,0,0,1,0.842010,High,High
4,-65.8477,47.9468,480,20.5,57.2,35.412586,56,197,6,71.6,...,0,0,0,0,1,0,0,0.471364,Medium,Medium


In [2]:
# ======================================
# RECONSTRUCT TRAFFIC LEVEL
# ======================================

def get_traffic_level(row):
    if row["Traffic_Status_Heavy"] == 1:
        return "Heavy"
    elif row["Traffic_Status_Detour"] == 1:
        return "Detour"
    else:
        return "Clear"

df["traffic_level"] = df.apply(get_traffic_level, axis=1)

df["traffic_level"].value_counts()

traffic_level
Detour    345
Clear     328
Heavy     327
Name: count, dtype: int64

In [3]:
# ======================================
# OPERATIONAL BASE TIME
# ======================================

operational_base_time = df["Waiting_Time"].mean()

print("Operational Base Time:", operational_base_time)

Operational Base Time: 35.062


In [4]:
# ======================================
# TRAFFIC IMPACT
# ======================================

traffic_impact = (
    df.groupby("traffic_level")["delay_probability"]
    .mean()
)

print("Traffic Impact (Average Delay Probability):")
print(traffic_impact)


Traffic Impact (Average Delay Probability):
traffic_level
Clear     0.460982
Detour    0.382093
Heavy     0.865091
Name: delay_probability, dtype: float64


In [ ]:
# ======================================
# BASELINE ETA
# ======================================

df["traffic_delay_factor"] = df["traffic_level"].map(traffic_impact)

df["baseline_eta"] = (
    operational_base_time
    + (df["traffic_delay_factor"] * operational_base_time)
)

df[["traffic_level", "baseline_eta"]].head()

,traffic_level,baseline_eta
0,Detour,48.458938
1,Heavy,65.393832
2,Detour,48.458938
3,Heavy,65.393832
4,Clear,51.224934


In [7]:
# ======================================
# Risk - Action Mapping
# ======================================

def get_action(row):

    risk = row["final_risk_level"]
    utilization = row["Asset_Utilization"]

    if risk == "Low":
        return "A_Normal"

    elif risk == "Medium":
        return "B_Monitor"

    elif risk == "High":
        if utilization > 90:
            return "D_Reroute_Notify_Redistribute"
        else:
            return "C_Reroute_Notify"

    elif risk == "Critical":
        return "D_Reroute_Notify_Redistribute"
    
df["action_taken"] = df.apply(get_action, axis=1)

df[["final_risk_level", "action_taken"]].head()

,final_risk_level,action_taken
0,Low,A_Normal
1,Critical,D_Reroute_Notify_Redistribute
2,High,D_Reroute_Notify_Redistribute
3,High,D_Reroute_Notify_Redistribute
4,Medium,B_Monitor


In [11]:
# =========================================
# DATA-DRIVEN REROUTE IMPROVEMENT
# =========================================

clear_factor = traffic_impact.min()  # best traffic condition
improvement_rate = 0.5  # recover 50% of congestion gap

def optimized_traffic_factor(row):
    
    if "Reroute" in row["action_taken"]:
        
        original = row["traffic_delay_factor"]
        
        improved = original - improvement_rate * (original - clear_factor)
        
        return improved
    
    else:
        return row["traffic_delay_factor"]


df["optimized_traffic_delay_factor"] = df.apply(optimized_traffic_factor, axis=1)

df[["final_risk_level",
    "traffic_delay_factor",
    "optimized_traffic_delay_factor"]].head()


,final_risk_level,traffic_delay_factor,optimized_traffic_delay_factor
0,Low,0.382093,0.382093
1,Critical,0.865091,0.623592
2,High,0.382093,0.382093
3,High,0.865091,0.623592
4,Medium,0.460982,0.460982


In [13]:
# =========================================
# OPTIMIZED ETA
# =========================================

df["optimized_eta"] = (
    operational_base_time
    + (df["optimized_traffic_delay_factor"] * operational_base_time)
)

df[[
    "traffic_level",
    "baseline_eta",
    "optimized_eta",
    "action_taken"
]].head()


,traffic_level,baseline_eta,optimized_eta,action_taken
0,Detour,48.458938,48.458938,A_Normal
1,Heavy,65.393832,56.926385,D_Reroute_Notify_Redistribute
2,Detour,48.458938,48.458938,D_Reroute_Notify_Redistribute
3,Heavy,65.393832,56.926385,D_Reroute_Notify_Redistribute
4,Clear,51.224934,51.224934,B_Monitor


In [14]:
# ============================================
# UTILIZATION IMPACT ANALYSIS
# ============================================

utilization_impact = (
    df.groupby(pd.cut(df["Asset_Utilization"], bins=[0,70,90,100]))
      ["delay_probability"]
      .mean()
)

print("Average Delay Probability by Utilization Bucket:")
print(utilization_impact)

Average Delay Probability by Utilization Bucket:
Asset_Utilization
(0, 70]      0.558088
(70, 90]     0.568351
(90, 100]    0.569689
Name: delay_probability, dtype: float64


C:\Users\Anish Mukul Ringe\AppData\Local\Temp\ipykernel_1224\3445141551.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.cut(df["Asset_Utilization"], bins=[0,70,90,100]))


In [17]:
print(utilization_impact.index)

CategoricalIndex([(0, 70], (70, 90], (90, 100]], categories=[(0, 70], (70, 90], (90, 100]], ordered=True, dtype='category', name='Asset_Utilization')


In [19]:
high_bucket = pd.Interval(90, 100, closed='right')
medium_bucket = pd.Interval(70, 90, closed='right')

high_util_factor = utilization_impact.loc[high_bucket]
medium_util_factor = utilization_impact.loc[medium_bucket]

stress_gap = high_util_factor - medium_util_factor

print("High Utilization Delay Factor:", high_util_factor)
print("Medium Utilization Delay Factor:", medium_util_factor)
print("Improvement Potential (Stress Gap):", stress_gap)

High Utilization Delay Factor: 0.5696887748630928
Medium Utilization Delay Factor: 0.5683513850152272
Improvement Potential (Stress Gap): 0.0013373898478655777
